In [1]:
import simpy
from numpy.random import choice
#from random import expovariate
import numpy as np

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number

In [3]:
file_name_full_graph='full_graph_{}_{}.txt'.format(n,N)
file_name_full_graph_optimal_path='full_graph_and_prime_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_graph='prime_graph_{}_{}.txt'.format(n,N)
file_name_prime_graph_optimal_path='prime_graph_optimal_path_{}_{}.txt'.format(n,N)
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)
file_name_cluster_graph='cluster_graph_{}_{}.txt'.format(n,N)

# Reading simple optimal path

In [4]:
full_graph_opt_path=[0]*(n+N+1)

for i in range(n+N+1):
    full_graph_opt_path[i]=[0]*(n+N+1)
    

In [5]:
with open('test/'+file_name_full_graph_optimal_path, 'r') as f:
    
    n,N=map(int,f.readline().split())

    line_cnt=0
    for line in f:
        line_cnt+=1 
        
        if line_cnt%4:
            
            if line_cnt%4==1:
                i,j=map(int,line.split())     

            if line_cnt%4==3:
                
                path = list(map(int,line.split()))
                
                #print('path',path)
                full_graph_opt_path[i][j]=path

    f.close()

In [6]:
full_graph_opt_path[2][7]

[2, 3, 7]

# Network simulation

In [7]:
class Packet(object):
    """ A very simple class that represents a packet.
        This packet will run through a queue at a switch output port.
        We use a float to represent the size of the packet in bytes so that
        we can compare to ideal M/M/1 queues.

        Parameters
        ----------
        
        packet_id : int
            an identifier for the packet
            
        path : list
            path from source to destination
            
        cur_path_ind : int
            current path position
        
        time : float
            the time the packet arrives at the output queue
            
        size : float
            the size of the packet in bytes
        
        src, dst : int
            identifiers for the source and the destination

    """
    def __init__(self, packet_id, path, cur_path_ind, time, size, src, dst):
        
        self.packet_id=packet_id
        self.path=path
        self.cur_path_ind=cur_path_ind
        
        
        self.time = time
        self.size = size
        self.src = src
        self.dst = dst


    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.packet_id, self.src, self.time, self.size)

In [8]:
class PacketGenerator(object):
    """ Generates packets with given inter-arrival time distribution. 
        Set the "out" member variable to the entity to receive the packet.
        Then it generates a destination (for each package independently)
        according to predetermined distribution and sends packages along the predetemined paths.
        PacketGenerator i is connected to Switch i.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
            
        gen_prob : list
            destination probability distribution
            
        generator_id : int
            PacketGenerator id
            
        adist : function
            a no parameter function that returns the successive inter-arrival times of the packets
            
        sdist : function
            a no parameter function that returns the successive sizes of the packets
            
        initial_delay : int
            Starts generation after an initial delay. Default = 0
            
        finish : int
            Stops generation at the finish time. Default is infinite


    """
    def __init__(self, env, gen_prob, generator_id, adist, sdist, initial_delay=0, finish=float("inf")):
        
        self.gen_prob=gen_prob
        self.generator_id = generator_id
        self.env = env
        self.adist = adist
        self.sdist = sdist
        self.initial_delay = initial_delay
        self.finish = finish
        self.packets_sent = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process


    def run(self):
        """The generator function used in simulations.
        """
        
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            yield self.env.timeout(self.adist())
            
            destination = choice(range(1,N+1), p=self.gen_prob)
            opt_path=full_graph_opt_path[self.generator_id][destination]
            
            #print('opt_path_generator, id, dest',opt_path, self.generator_id, destination)

            self.packets_sent += 1
            
            pkt = Packet(packet_id=self.packets_sent, 
                         path=opt_path, 
                         cur_path_ind=0, 
                         time=self.env.now, 
                         size=self.sdist(), 
                         src=self.generator_id, 
                         dst=destination)

            
            self.out.put(pkt)
       

In [9]:
class PacketSink(object):
    """ Receives packets and collects delay information into the
        waits list. You can then use this list to look at delay statistics.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
            
        debug : boolean
            if true then the contents of each packet will be printed as it is received.
            
        rec_arrivals : boolean
            if true then arrivals will be recorded
            
        absolute_arrivals : boolean
            if true absolute arrival times will be recorded, otherwise the time between consecutive arrivals
            is recorded.
            
        rec_waits : boolean
            if true waiting time experienced by each packet is recorded
            

    """
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False):
        self.store = simpy.Store(env)
        self.env = env
        self.rec_waits = rec_waits
        self.rec_arrivals = rec_arrivals
        self.absolute_arrivals = absolute_arrivals
        self.waits = []
        self.arrivals = []
        self.debug = debug
        self.packets_rec = 0
        self.bytes_rec = 0
        self.last_arrival = 0.0

    def put(self, pkt):

        now = self.env.now
        
        if self.rec_waits:
            self.waits.append(self.env.now - pkt.time)
        if self.rec_arrivals:
            if self.absolute_arrivals:
                self.arrivals.append(now)
            else:
                self.arrivals.append(now - self.last_arrival)
            self.last_arrival = now
        self.packets_rec += 1
        self.bytes_rec += pkt.size


In [10]:
class SwitchPort(object):
    """ Models a switch output port with a given rate and buffer size limit in bytes.
        First, it stores packets in queue (put method), then it extracts them one-by-one 
        and sends to the next switch or sink.
        

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        rate : float
            the bit rate of the port
        qlimit : integer (or None)
            a buffer size limit in bytes or packets for the queue (including items
            in service).
        limit_bytes : If true, the queue limit will be based on bytes if false the
            queue limit will be based on packets.

    """
    def __init__(self, env, router_id, rate, qlimit=None, limit_bytes=True, debug=False):
        self.router_id=router_id
        self.store = simpy.Store(env)
        self.rate = rate
        self.env = env
        self.out = None
        self.packets_rec = 0
        self.packets_drop = 0
        self.qlimit = qlimit
        self.limit_bytes = limit_bytes
        self.byte_size = 0  # Current size of the queue in bytes
        self.debug = debug
        self.busy = 0  # Used to track if a packet is currently being sent
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    def run(self):

        while True:
            msg = (yield self.store.get())
            self.busy = 1
            self.byte_size -= msg.size
            yield self.env.timeout(msg.size*8.0/self.rate)
            
            path=msg.path
            cur_path_ind=msg.cur_path_ind
            destination=msg.dst
                
            print('path',path)
                
            if path[cur_path_ind]==destination:
                
                sink.put(msg)
            else:
                next_node=path[cur_path_ind+1]
                msg.cur_path_ind=cur_path_ind+1
                
                #print('',)
                #print('next_node',next_node)
                #print('switch[next_node]',switch[next_node])
                switch[next_node].store.put(msg)   
                
                
                
            self.busy = 0
            if self.debug:
                print(msg)
                
     

    def put(self, pkt):

        self.packets_rec += 1
        tmp_byte_count = self.byte_size + pkt.size

        if self.limit_bytes and tmp_byte_count >= self.qlimit:
            self.packets_drop += 1
            return
        
        else:
            self.byte_size = tmp_byte_count
            
            return self.store.put(pkt)    
                

In [11]:
def RandomizedArrival():
    #return 1.5    # time interval
    return max(np.random.normal(1.5, 0.5),1)

def constSize():
    return 100.0  # bytes

In [12]:
gen_prob=[0]*(N+1)

for i in range(1,N+1):
    gen_prob[i]=[1./(N-1) if j+1!=i else 0 for j in range(N)]
    


In [13]:
gen_prob

[0,
 [0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111],
 [0.1111111111111111,
  0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111],
 [0.1111111111111111,
  0.1111111111111111,
  0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111],
 [0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111],
 [0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,

In [14]:
generators=[0]*(N+1)
switch=[0]*(n+N+1)

env = simpy.Environment()  # Create the SimPy environment

sink=PacketSink(env, debug=True)

for i in range(1,n+N+1):
    switch[i]=SwitchPort(env=env, router_id=i, rate=1000.0, qlimit=3000, debug=True)

for i in range(1,N+1):
    generators[i]=PacketGenerator(env, gen_prob[i], i, RandomizedArrival, constSize)
    generators[i].out = switch[i]


    
env.run(until=2000)
print("average delivery time:", format(sum(sink.waits)/max(1,sink.packets_rec)))
print('number of delivered packages:', sink.packets_rec)
print('number of packages stuck in the network:\n')

for i in range(1,n+N+1):
    print('router {} packages {}'.format(i,len(switch[i].store.items)))

print('\n')
for i in range(1,n+N+1):
    print("router {} has dropped {} packages ".format(i,switch[i].packets_drop))

print('\n')
for i in range(1,N+1):        
    print('generator {} has sent {} packages'.format(i, generators[i].packets_sent))


path [7, 13, 5]
id: 1, src: 7, time: 1, size: 100.0
path [10, 6, 3, 2]
id: 1, src: 10, time: 1, size: 100.0
path [9, 4]
id: 1, src: 9, time: 1.043919591343052, size: 100.0
path [3, 6, 14, 9, 4]
id: 1, src: 3, time: 1.2168510998358153, size: 100.0
path [4, 9, 13, 8]
id: 1, src: 4, time: 1.334282969679243, size: 100.0
path [5, 13, 9, 10]
id: 1, src: 5, time: 1.3448855989023965, size: 100.0
path [2, 3, 6, 10]
id: 1, src: 2, time: 1.5221282739769533, size: 100.0
path [7, 13, 5]
id: 1, src: 7, time: 1, size: 100.0
path [10, 6, 3, 2]
id: 1, src: 10, time: 1, size: 100.0
path [8, 13, 9]
id: 1, src: 8, time: 1.9935423687236125, size: 100.0
path [9, 14, 6]
id: 2, src: 9, time: 2.0439195913430517, size: 100.0
path [1, 3, 6, 10]
id: 1, src: 1, time: 2.112658524470155, size: 100.0
path [9, 4]
id: 1, src: 9, time: 1.043919591343052, size: 100.0
path [2, 3, 6, 10]
id: 1, src: 2, time: 1.5221282739769533, size: 100.0
path [10, 9, 13, 7]
id: 2, src: 10, time: 2.33319824036528, size: 100.0
path [5, 13,

path [10, 6]
id: 26, src: 10, time: 39.0540621210107, size: 100.0
path [1, 13, 5]
id: 30, src: 1, time: 47.13965774825598, size: 100.0
path [6, 7]
id: 25, src: 6, time: 39.11784831525451, size: 100.0
path [6, 14, 9]
id: 26, src: 6, time: 40.11784831525451, size: 100.0
path [2, 9]
id: 16, src: 2, time: 26.1002441279124, size: 100.0
path [9, 13, 5]
id: 8, src: 9, time: 11.953640982313377, size: 100.0
path [10, 6, 3, 1]
id: 17, src: 10, time: 24.27281265751421, size: 100.0
path [2, 3, 1, 13, 8]
id: 30, src: 2, time: 50.417811830937445, size: 100.0
path [4, 9, 10]
id: 36, src: 4, time: 50.51373404677448, size: 100.0
path [1, 13, 8]
id: 15, src: 1, time: 23.492719388782586, size: 100.0
path [6, 3, 1]
id: 27, src: 6, time: 41.74298139845299, size: 100.0
path [3, 1, 13, 8]
id: 21, src: 3, time: 31.036208339007516, size: 100.0
path [2, 3, 7]
id: 19, src: 2, time: 30.796103473204678, size: 100.0
path [8, 13, 7]
id: 32, src: 8, time: 50.623180328280654, size: 100.0
path [10, 6, 3, 2]
id: 33, src

path [5, 13, 9, 4]
id: 66, src: 5, time: 98.95540830788548, size: 100.0
path [10, 9, 13, 5]
id: 66, src: 10, time: 99.11990140701451, size: 100.0
path [3, 1]
id: 42, src: 3, time: 65.48834610599476, size: 100.0
path [2, 3, 7]
id: 60, src: 2, time: 98.215318144134, size: 100.0
path [4, 9, 13, 5]
id: 16, src: 4, time: 23.26492380896217, size: 100.0
path [3, 6]
id: 34, src: 3, time: 52.855352021853925, size: 100.0
path [8, 13, 1, 3]
id: 27, src: 8, time: 41.654863568587125, size: 100.0
path [8, 13, 7, 6]
id: 28, src: 8, time: 43.50181796708088, size: 100.0
path [4, 9, 13, 1]
id: 68, src: 4, time: 98.97832314383628, size: 100.0
path [6, 14, 9]
id: 26, src: 6, time: 40.11784831525451, size: 100.0
path [6, 10]
id: 52, src: 6, time: 83.87421855884209, size: 100.0
path [10, 6, 3, 1]
id: 35, src: 10, time: 53.50274381166952, size: 100.0
path [2, 9]
id: 61, src: 2, time: 99.65117968750572, size: 100.0
path [8, 13, 1]
id: 63, src: 8, time: 100.05491690739001, size: 100.0
path [1, 13, 7]
id: 28, s

id: 52, src: 9, time: 79.14906628251094, size: 100.0
path [2, 3]
id: 59, src: 2, time: 97.215318144134, size: 100.0
path [4, 9, 10]
id: 53, src: 4, time: 77.59027776940508, size: 100.0
path [1, 13, 7]
id: 43, src: 1, time: 65.25288827682265, size: 100.0
path [10, 9, 13, 7]
id: 24, src: 10, time: 36.14956516767808, size: 100.0
path [10, 6]
id: 82, src: 10, time: 125.01605186856622, size: 100.0
path [1, 3]
id: 96, src: 1, time: 149.89219141513396, size: 100.0
path [7, 13, 5]
id: 43, src: 7, time: 65.15032655287577, size: 100.0
path [10, 6, 3, 2]
id: 54, src: 10, time: 80.8658716141226, size: 100.0
path [10, 9, 13, 5]
id: 52, src: 10, time: 78.35895384129148, size: 100.0
path [8, 13, 9, 10]
id: 98, src: 8, time: 152.46428970260587, size: 100.0
path [10, 6, 3, 1]
id: 55, src: 10, time: 82.52420872807303, size: 100.0
path [9, 10]
id: 52, src: 9, time: 79.14906628251094, size: 100.0
path [7, 13, 9]
id: 101, src: 7, time: 149.9498296485172, size: 100.0
path [3, 1, 13, 8]
id: 27, src: 3, time:

id: 133, src: 5, time: 205.40905681104616, size: 100.0
path [10, 6, 3]
id: 70, src: 10, time: 105.98765631590335, size: 100.0
path [3, 1, 13, 8]
id: 81, src: 3, time: 128.71636413565983, size: 100.0
path [10, 6, 3, 2]
id: 132, src: 10, time: 205.66008973427498, size: 100.0
path [7, 6]
id: 133, src: 7, time: 204.50184600211907, size: 100.0
path [10, 9, 13, 7]
id: 31, src: 10, time: 47.2462582502037, size: 100.0
path [6, 7]
id: 109, src: 6, time: 173.6436213961594, size: 100.0
path [8, 13, 9]
id: 131, src: 8, time: 204.36389270482152, size: 100.0
path [9, 13, 8]
id: 72, src: 9, time: 108.53722380950009, size: 100.0
path [4, 9, 13, 8]
id: 138, src: 4, time: 206.0804330638188, size: 100.0
path [3, 7]
id: 82, src: 3, time: 130.2998365087573, size: 100.0
path [2, 3, 1, 13, 8]
id: 79, src: 2, time: 127.51917064075943, size: 100.0
path [2, 3]
id: 130, src: 2, time: 206.44046213676967, size: 100.0
path [8, 13, 7]
id: 58, src: 8, time: 91.50630138710584, size: 100.0
path [7, 13, 1]
id: 60, src: 

...

path [8, 13, 7]
id: 519, src: 8, time: 802.6301905186398, size: 100.0
path [4, 9, 14, 6, 3]
id: 493, src: 4, time: 763.2629729623195, size: 100.0
path [7, 6]
id: 1159, src: 7, time: 1786.5879988663696, size: 100.0
path [6, 14, 9, 4]
id: 924, src: 6, time: 1427.6451570644122, size: 100.0
path [1, 13, 9, 4]
id: 278, src: 1, time: 419.62019569971534, size: 100.0
path [4, 9, 2]
id: 619, src: 4, time: 953.9817413697829, size: 100.0
path [8, 13, 7]
id: 1149, src: 8, time: 1787.6679968071726, size: 100.0
path [10, 6, 3, 2]
id: 1173, src: 10, time: 1787.0183600331104, size: 100.0
path [10, 6, 3, 2]
id: 551, src: 10, time: 843.0391741095989, size: 100.0
path [1, 3, 6]
id: 1153, src: 1, time: 1773.1948662537861, size: 100.0
path [6, 7, 13, 8]
id: 416, src: 6, time: 654.8348435030603, size: 100.0
path [6, 7]
id: 925, src: 6, time: 1428.6451570644122, size: 100.0
path [8, 13, 7]
id: 519, src: 8, time: 802.6301905186398, size: 100.0
path [5, 13, 9]
id: 1172, src: 5, time: 1788.2924825445891, size: 

path [3, 1]
id: 702, src: 3, time: 1089.5258639765589, size: 100.0
path [1, 3, 6]
id: 1187, src: 1, time: 1828.6610081883305, size: 100.0
path [8, 13, 1]
id: 533, src: 8, time: 826.0822982817132, size: 100.0
path [3, 6, 10]
id: 557, src: 3, time: 869.4590261436265, size: 100.0
path [3, 6, 14, 9, 4]
id: 556, src: 3, time: 868.2444621110561, size: 100.0
path [6, 10]
id: 953, src: 6, time: 1469.2814280769082, size: 100.0
path [7, 13, 9, 10]
id: 1191, src: 7, time: 1838.2889961111605, size: 100.0
path [2, 3, 6]
id: 1198, src: 2, time: 1840.3206501237566, size: 100.0
path [8, 13, 9, 4]
id: 281, src: 8, time: 442.30642506135007, size: 100.0
path [4, 9, 14, 6, 3]
id: 1199, src: 4, time: 1839.689744538669, size: 100.0
path [5, 13, 1]
id: 1206, src: 5, time: 1840.5755651182299, size: 100.0
path [2, 3, 6, 10]
id: 711, src: 2, time: 1088.8078834338498, size: 100.0
path [5, 13, 1, 3]
id: 533, src: 5, time: 821.0880321028142, size: 100.0
path [10, 9, 13, 5]
id: 284, src: 10, time: 441.0224642573341

path [9, 14, 6, 3]
id: 314, src: 9, time: 474.0098518946297, size: 100.0
path [10, 6, 3, 1]
id: 575, src: 10, time: 881.6423563758649, size: 100.0
path [8, 13, 7, 6]
id: 1215, src: 8, time: 1893.6281993523064, size: 100.0
path [8, 13, 5]
id: 548, src: 8, time: 851.0979797552424, size: 100.0
path [3, 6]
id: 571, src: 3, time: 889.3494821401368, size: 100.0
path [6, 7]
id: 977, src: 6, time: 1506.329027944971, size: 100.0
path [10, 9, 4]
id: 1238, src: 10, time: 1893.4000263606283, size: 100.0
path [4, 9, 14, 6, 3]
id: 1236, src: 4, time: 1893.8736317529176, size: 100.0
path [5, 13, 9]
id: 298, src: 5, time: 455.2238693632746, size: 100.0
path [2, 3, 1, 13, 5]
id: 1233, src: 2, time: 1892.9971359853778, size: 100.0
path [5, 13, 7]
id: 1241, src: 5, time: 1894.26140926768, size: 100.0
path [3, 6, 10]
id: 723, src: 3, time: 1122.6744730852301, size: 100.0
path [1, 13, 9, 4]
id: 1220, src: 1, time: 1878.3310397932771, size: 100.0
path [3, 1, 13, 8]
id: 328, src: 3, time: 509.9641864103234, 

path [6, 7]
id: 1005, src: 6, time: 1547.3822327121584, size: 100.0
path [10, 6, 3, 2]
id: 594, src: 10, time: 910.78802178771, size: 100.0
path [2, 3]
id: 1263, src: 2, time: 1943.3238516830697, size: 100.0
path [1, 13, 7]
id: 1252, src: 1, time: 1929.0466131324206, size: 100.0
path [8, 13, 1, 3, 2]
id: 563, src: 8, time: 875.1462555241006, size: 100.0
path [10, 6, 3, 1]
id: 1014, src: 10, time: 1546.4817149835892, size: 100.0
path [3, 6, 14, 9]
id: 322, src: 3, time: 501.48000966540957, size: 100.0
path [9, 14, 6, 3]
id: 680, src: 9, time: 1042.3574994464957, size: 100.0
path [7, 13, 9, 10]
id: 1259, src: 7, time: 1944.4719854727125, size: 100.0
path [5, 13, 9, 10]
id: 1274, src: 5, time: 1944.6365404999533, size: 100.0
path [2, 3, 7]
id: 754, src: 2, time: 1150.8116025049867, size: 100.0
path [2, 3, 1]
id: 1264, src: 2, time: 1944.3238516830697, size: 100.0
path [1, 13, 8]
id: 1253, src: 1, time: 1930.0883691974364, size: 100.0
path [8, 13, 1, 3]
id: 1247, src: 8, time: 1944.8684373

path [4, 9, 14, 6, 3]
id: 694, src: 4, time: 1072.149937759495, size: 100.0
path [7, 13, 5]
id: 1293, src: 7, time: 1996.0927725970241, size: 100.0
path [3, 1, 13, 5]
id: 763, src: 3, time: 1186.0987183865366, size: 100.0
path [1, 3, 6, 10]
id: 1286, src: 1, time: 1982.1346985576113, size: 100.0
path [8, 13, 9, 10]
id: 308, src: 8, time: 483.20796355756494, size: 100.0
path [8, 13, 9, 10]
id: 1281, src: 8, time: 1998.559216111041, size: 100.0
path [5, 13, 7]
id: 585, src: 5, time: 900.330978414719, size: 100.0
path [6, 3, 1]
id: 1030, src: 6, time: 1586.9174419120332, size: 100.0
path [2, 3, 1, 13, 8]
id: 1299, src: 2, time: 1998.732129766821, size: 100.0
path [9, 2]
id: 700, src: 9, time: 1073.4600875272924, size: 100.0
path [4, 9, 14, 6, 3]
id: 694, src: 4, time: 1072.149937759495, size: 100.0
path [7, 13, 9]
id: 1294, src: 7, time: 1997.5586476157553, size: 100.0
path [6, 3, 2]
id: 601, src: 6, time: 933.4929656559397, size: 100.0
path [1, 3]
id: 1287, src: 1, time: 1983.66332138838